In [9]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.isri import ISRIStemmer
import spacy
import string
# from google.cloud import vision, translate
# from google.cloud.vision import types
# import google.auth
# from protobuf_to_dict import protobuf_to_dict
# from google.protobuf.json_format import MessageToJson
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import gensim
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full

from collections import OrderedDict
from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import row, column
import random


import importlib
ldavis = importlib.util.find_spec("pyLDAvis")
import pyLDAvis.gensim as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
# print(pyLDAvis.enable_notebook())

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
ar_df = pd.read_csv('arabic2_test.csv', names=['file_name', 'page_number', 'google_vision_text', 'google_translate_text'])

In [4]:
ar_df

,file_name,page_number,google_vision_text,google_translate_text
0,image_lucida_app/media/Data_RepositoryEgyptian...,0,العدد ۱۷۷۲ - السنة السابعة والثلاثون\nالاثنين...,Number 1772 - Thirty-seventh year Monday 28 Ma...
1,image_lucida_app/media/Data_RepositoryEgyptian...,4,دولارا لیں صودا\nالنكون عون القوالبراعم الواه...,"3 - clarity in the vision of goals, and follow..."
2,image_lucida_app/media/Data_RepositoryEgyptian...,5,اللا کے\nت روما\nعضو اللجنة السياسية\nعزیز صد...,Al-Lajka Rome Member of the Political Committe...
3,image_lucida_app/media/Data_RepositoryEgyptian...,6,تمرير الخبراء الأبد\nمشروعات والعب في البر لز...,Experts forever pass projects and play on land...
4,image_lucida_app/media/Data_RepositoryEgyptian...,9,5\n*\nميثاق الملك حسين !! .\nويشكل خلال الأسا...,5 * The Charter of King Hussein !! . Within th...
5,image_lucida_app/media/Data_RepositoryEgyptian...,1,ملاحظات حول الميثان)\nاقوا النار على مستوى ال...,Notes about methane) Fire the level of Constit...
6,image_lucida_app/media/Data_RepositoryEgyptian...,10,رویه\nا\nكيف تقرأ الميثاق.\nی\nعم السعرالسای\...,See how to read the Charter. The price of the ...
7,image_lucida_app/media/Data_RepositoryEgyptian...,11,-\nفي هااطی ۰.را نفت قانون اراصاع الزای\nهل ه...,- In Hatti 0.37 The law of Ara&#39;aa al-Zayi ...
8,image_lucida_app/media/Data_RepositoryEgyptian...,3,بهیارفل دلکویرالطوارجاعه\nالى حد انیبال لبنان...,"No, certainly not .. And it happened that the ..."
9,image_lucida_app/media/Data_RepositoryEgyptian...,12,أحكااال\nایزنی\nفي تمام الثامنة من مساء الاثن...,"Ahkala Eizni At 8:00 pm last Monday, a major e..."


In [5]:
st = ISRIStemmer()
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()
    print('text', st.stem(text))
    return text

def custom_tokenize(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text(df):
    doc = []
    final_doc = []
    for index, row in df.iterrows():
        raw_text = row['google_vision_text']
        tokens = clean_str(raw_text)
        text = custom_tokenize(tokens)
        page_terms = ''
        for t in text: 
            if t in stopwords.words('arabic'):
                print('stopword', t)
            else:
#                 print(t)
                page_terms += t + ' '
            doc.append(page_terms)
#     print(doc)
    return doc


    

In [6]:
texts = process_text(ar_df)


text عدد ۱۷۷۲   السنه السابعه والثلاثون الاثنين ۲۸ مایو س ن ه ۱۹۹۲ الثمن 40 مليما 52 ه M | 2  هم ( الا الميثاق الوطني المهندس: جمال عبد الناص O   الباشمهندس جاب لنا الرسم التفصيلي بتاع الفيلا  !
stopword هم
stopword لنا
text دولارا لیں صودا النكون عون القوالبراعم الواها للظروف المتغيره التي يجابهها النضال العربي  علي ان تلتزم هذه الحركه باهداف النضال وبمثله الاحلافيه  ٣  الوضوح في رؤيه الاهداف  ومتابعتها باستمرار  وتجنب الانسياق الانفعالي الي الدروب الفرعيه التي تبتعد بالنضال الوطني عن طريقه وتهدد جزءا كبيرا من طاقته ۰۰ هذه البنود الثلاثه لا تحدد المطالبه الوطنيه الاساسيه فحسب  بل تحدد مباديء الفلسفه الثوريه  والنظريه الثوريه  وتحدد مبادی الخطه العامه ای والاستراتيجيه  ثم تحدد ایفا مباديء الخطه التفصيليه اي «التاكتيك ) التوری ۰۰ ولكنها في كل ذلك لم تخرج عن المباديء  لم تمتد الي التفاصيل الجزئيه في تطبيقاتها ۰۰ بل ان الميثاق وصل في تحديد المباديء الي ابعد من هذا ۰۰ وصل الي جزئیات بعيده  مثلا  عندما يحدث الميثاق عن اداره الانتاج قال : «  انه من الزم الامور منا تش ج يع الكلمه المكتوبه لتك

stopword الذي
stopword مع
stopword التي
stopword بها
stopword التي
stopword هذه
stopword الذين
stopword في
stopword بين
stopword في
stopword في
stopword التي
stopword في
stopword عن
stopword هذا
stopword من
stopword هذا
stopword من
stopword ذلك
stopword الذي
stopword في
stopword هذا
stopword من
stopword قد
stopword من
stopword بعض
stopword مع
stopword ولكن
stopword هذه
stopword قد
stopword مع
stopword في
stopword ذه
stopword في
stopword هو
stopword مما
stopword من
stopword في
stopword من
stopword في
stopword في
stopword من
stopword مع
stopword التي
stopword بها
stopword في
stopword في
stopword من
stopword من
stopword عليه
stopword التي
stopword هؤلاء
stopword الذين
stopword به
stopword في
stopword من
stopword الذين
text 5 * ميثاق الملك حسين  !  !   ويشكل خلال الاسابيع الثلاثه القادمه  عدد من اللجان الهامه ۰۰ و استدعي احد س فراء الجمهوريه العربيه المتحده  باحدي العواصم الهامه  للتشاور ۰۰ مده تشاور القاهره  مع وشیك بثلاثه ملاين جنيه السفير المستمتعي  غير محدده : تسلمه وزير الاقتص ا د من 

stopword هل
stopword هناك
stopword لم
stopword من
text بهیارفل دلکویرالطوارجاعه الي حد انیبال لبنان عن المبرملا کارل مار کس ؛  لا  قطعا لا  والذي حدث ان ص ياغه الميثاق قد حررت التعاريف القديمه من معانيها القديمه  حررتها من الاحتكار المذهبی  واستعملتها بمعان جديده  !  ۱۰ وقد كانت بعض المذاهب تحتكر لنفسها بعض التعاريف  کانت کيمه « السلام  تحتكرها الشيوعيه  وبلغ من قوه احتكرها ان اعجب كل من يستعمل ش عار , السلام  او ينضم لمنظمه تعمل من اجل السلام  يعتبر شيوعا  ثم استطاع جمال عبد الناصر ان يحطم هذا الاحتكار  وان يجعل من «السلام دعوه انسانيه  لا دعوه شيوعيه  اصبح كل فرد الان يستطيع ان يدعو للسلام  وهو مطمئن الي انه لن يتهم بالشيوعيه  ورغم التحديد القاطع لاشتراکیتنا في اكثر من موضع من الميثاق  وهو التحديد الذي يبعدها عن اشتراكيه کارل مارکس  وعن الفسفه الماديه المحنه ۰۰ رغم دياك  فان جمال عبد الناصر عندما استعمل تعریف والاشتراكيه العلميه  مهد لهنا التعريف فيلا : ان هناك تجارب اخري للتقدم  حققت اهدافها علي حساب زياده شقاء الشعب العامل واستغلاله اما لصالح راس المال  او تحت صوط تطبيقات مدهبيه مض

stopword فيه
stopword في
stopword عن
stopword ذلك
stopword بعد
stopword هذا
stopword بين
stopword هناك
stopword في
stopword التي
stopword عن
stopword في
stopword بين
stopword التي
stopword لا
stopword كل
stopword ذلك
stopword التي
stopword عن
stopword فيها
stopword في
stopword من
stopword بعد
stopword ذلك
stopword ماذا
stopword هنالك
stopword التي
stopword لم
stopword في
stopword ثم
stopword لا
stopword في
stopword هذا
stopword هو
stopword كل
stopword ذلك
stopword هناك
stopword بين
stopword بين
stopword عن
stopword لها
stopword منه
stopword في
stopword كل
stopword هذا
stopword هو
stopword هذا
stopword هناك
stopword كل
stopword لا
stopword في
stopword لن
stopword ذلك
stopword ما
stopword من
stopword هذه
stopword من
stopword في
stopword هذه
stopword مما
stopword غير
stopword في
stopword في
stopword في
stopword في
stopword والذي
stopword هناك
stopword من
stopword في
stopword كل
stopword من
stopword في
stopword كل
stopword لا
stopword منه
stopword لا
stopword في
stopword من
stopword هو
sto

stopword لا
stopword هل
stopword هؤلاء
stopword مع
text ي ان الامه • ՄՆԱԼ النقعيه )   ايوه ياكسلانس دي هي اللي ماش ماهي النقابات ي ه  !  !  لازم نطرد نمره (۲) عشان الصيف يتعدل || من الذي يراقب الجمعيات التعاونيه | جوده بين المنت : زمالك کے چالان تطهيره لان بحوالي 100 الف جنيه سویا ۰۰ ارشاد خلف الله ( نجع حمادي ): هناك مشكله اخري  تفتيت الملكات الصغيره 00 واحد يملك ندا نين علي خمس قطع  واحده عنده 16 قيراط علي اربع قطع  هذا التفتيت يطيع الانساج و بعله  ولا بد من تعويم الدورات س رقت الجمعيات وامتنع البنك عن تونلها الزراعيه  لعلاج هذا التفتيت  الحاج حسين محمود ( قنا): لا يزال هناك نهم مدحون في التسهر یں الزراعی  انا مثلا عددی بلح  ونلاح البحيره كامل زهري : اعتقد اننا نبدا بعيوب وبالنعل تم طرد بعض المستاجرين من كل جمعيه تعاونيه زراعيه لكبار الان عنده بطاطس  فانا اريد ان اعطيه التطبيق في الجمعيات التعاونيه  الاراضي التي يزرعونها ۰۰ بعد ان قدم والاربع اخماس البانيه لصغار الزراع من م ولی نیه لیدی من بحصوله  عبد الحي ابو انطا ( منوفيه ): الملاك لنقغماء اوراقا موثقه بامض اء علي ان يكون احد اعضاء

...

stopword به
stopword في
stopword عن
stopword بين
stopword عن
stopword هذه
stopword في
stopword من
stopword وهو
stopword في
stopword من
stopword من
stopword من
stopword له
stopword يا
stopword من
stopword في
stopword في
stopword في
stopword من
text دقيقتين تفكير باي قوه تحققت هذه المعجزات هو بالقوه الذاتيه التي في الاشتراكيه و بالطاقه المختزنه في المجاميم 40   في الاذرع العاطله  الاشتراكيه كانت امس تشغيل لكل هؤلاء  جعل المجتمع ينتفض علي قدميه ويتحول الي جيش عامل بینی و يفكر ويتفنن وبصمه ع ویزدع  وهذه هي الطاقه الاشتراكيه التي اطلقها الميثاق  اعلان حروق الانسان وقد كان هذا الميثاق التاريخي الشر من میثاق  كان عريضه ش امله بحقوق الانس ان  واعلانا خالد بقداسه هذه الحقوق و اعلانه بحريه الكلمه ادب واحترام الفرد اطر هم واعلانا بسياده القانون وه واعلانا بسلطه الشس يه رقيبه علي اجهزه الدوله » من واقعنا وروحنا تنبع شام الاشتراكيه لا من الكتبيه ولا من الفواكه ** الساثه التي في يدك تدرد لان فيها زمالك من القلب فيه قوه ذاتيه بحيث اذا ابرنمنه علي عدد من اللفت فانه يهود بقوه ذاتيه فيه فينفرج من تلقاء 

stopword الذي
stopword لا
stopword في
stopword تلك
stopword التي
stopword التي
stopword في
stopword ما
stopword التي
stopword من
stopword في
stopword من
stopword الذين
stopword في
stopword لا
stopword ومن
stopword ذلك
stopword في
stopword ليس
stopword ومن
stopword لا
stopword لم
stopword التي
stopword فيها
stopword غير
stopword لم
stopword من
stopword ذه
stopword التي
stopword لا
stopword لا
stopword في
stopword في
stopword في
stopword هذه
stopword لا
stopword به
stopword في
stopword هل
stopword التي
stopword من
stopword في
stopword مع
stopword كلما
stopword من
stopword في
stopword هناك
stopword في
stopword ما
stopword في
stopword من
stopword قد
stopword لا
stopword من
stopword ثم
stopword عن
stopword هؤلاء
stopword التي
stopword من
stopword لها
stopword عند
stopword في
stopword لها
stopword ثم
stopword من
stopword لا
stopword في
stopword بعد
stopword ذلك
stopword في
stopword التي
stopword من
text اصنع مستقبلك راضي امريك حياه انها الطمانينه بالادخار انهم ينكرون التصوف  وانا رجل تصوف ۰۰

stopword بين
stopword في
stopword مع
stopword في
stopword هذه
stopword له
stopword من
stopword مع
stopword كل
stopword التي
stopword من
stopword لها
stopword عن
stopword من
stopword به
stopword في
stopword من
stopword حيث
stopword في
stopword من
stopword ومن
stopword كل
stopword بما
stopword في
stopword ذلك
stopword هناك
stopword من
stopword الذين
stopword في
stopword من
stopword من
stopword هذه
stopword لا
stopword بما
stopword فيها
stopword كما
stopword من
stopword فلا
stopword من
stopword في
stopword في
stopword في
stopword كل
stopword في
stopword ولا
stopword من
stopword في
stopword لهم
stopword في
stopword لو
stopword ولو
stopword عن
stopword الذي
stopword بها
stopword لا
stopword فلا
stopword في
stopword هذا
stopword غير
stopword به
stopword في
stopword منه
stopword في
stopword في
stopword منه
stopword كل
stopword من
stopword بعد
stopword ذلك
stopword ما
stopword في
stopword في
stopword كل
stopword ما
stopword ها
stopword لا
stopword عن
stopword من
stopword ليس
stopword من
stopwo

stopword هو
stopword في
stopword من
stopword من
stopword الذي
stopword في
stopword التي
stopword من
stopword بين
stopword في
stopword مع
stopword في
stopword بين
stopword في
stopword بين
stopword في
stopword في
stopword بين
stopword في
stopword عند
stopword لا
stopword من
stopword الذي
stopword من
stopword في
stopword عن
stopword في
text مساحات ها و میدیا عادل دار السالسو فين الاوم   نکلے اما حارس مرمي النادي  وهو عملاق اوروبا  فانه يمسك الكره دائما بيد واحده  ويعتبر من احسن حارس المرمي في اوروبا ۰ وقد لعب فريق بنفيكا س يع عشره مباراه دوليه عام ۱۹۹۱ فاز فيها جميعا  واستطاع ان يحتل مكانه ريال مدريد في العالم  ويرفع سعر لاعبيه اضعافا مضاعفه  في خلال عام هذا هو فریق بنفيكا الذي انتزع النصر من فريق ريال مدريد المانی ۰۰ فكيف سيستقبله لاعبونا  ؟  ان لاعبي النادي الاهلي يس تعدون لهذه المباراه بروح عاليه هو صالح سليم يقول انه سلسب مدم المباراه ثم يفكر في اعتزال الكره في اما رفعت الفناجيلي فيقول بالثقه التي يلعب بها الكره : انه سيوجه نظراوروبا الي لاعبي افريقيا , ایام قلائل ستشهد القاهره  اروع 

stopword في
stopword في
stopword ولا
stopword من
stopword هذه
stopword في
stopword هذا
stopword في
stopword له
stopword في
stopword ليسوا
stopword به
stopword هذا
stopword لم
stopword في
stopword لن
stopword في
stopword ذلك
stopword التي
stopword بل
stopword في
stopword هذه
stopword منذ
stopword الذي
stopword في
stopword في
stopword من
stopword هذا
stopword الذين
stopword في
stopword من
stopword في
stopword عن
stopword ليس
stopword من
stopword هذه
stopword هي
stopword في
stopword الذي
stopword في
stopword في
stopword والذي
stopword له
stopword في
stopword الذي
stopword به
stopword من
stopword في
stopword من
stopword والذين
stopword في
stopword ثم
stopword من
stopword من
stopword من
stopword هذه
stopword الذي
stopword دون
stopword الذي
stopword من
stopword لا
stopword من
stopword التي
stopword ذه
stopword من
stopword بها
text يوم ** لا لارا های کیف ما معا و به و گاه عامل مه گه و راک وچهارم جز و گیر کشت فراسو مربا او را الكتاب الذهبي حادث النصف متر بقلم صبری دوسی يصدر اول يونيه عن مؤسسمي

In [18]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [20]:
set(nltk.corpus.stopwords.words("arabic"))

{'آه',
 'آها',
 'آي',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'أم',
 'أما',
 'أن',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنى',
 'أو',
 'أولئك',
 'أولاء',
 'أوه',
 'أي',
 'أين',
 'أينما',
 'أيها',
 'إذ',
 'إذا',
 'إذما',
 'إذن',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إما',
 'إن',
 'إنا',
 'إنما',
 'إنه',
 'إي',
 'إيه',
 'التي',
 'الذي',
 'الذين',
 'اللائي',
 'اللاتي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بيد',
 'بين',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل',
 'على',
 'عليك',
 'عليه',
 'عم

In [7]:
def custom_tokenize_english(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text_english(df, types):
    doc = []
    final_doc = []
    for index, row in df.iterrows():
        raw_text = row['google_translate_text']
        tokens = custom_tokenize_english(raw_text)
        page_terms = ''
        for t in tokens:
            if t in string.punctuation:
                pass
            elif t in stopwords.words('english'):
                pass
            else:
                page_terms += t.lower() + ' '
        doc.append(page_terms)
    print(doc)
    for sent in doc:
        sent_terms = ''
        spacy_text = nlp(sent)
        for ent in spacy_text.ents:
            if ent.label_ in types:
                sent_terms += ent.text + ' '
        final_doc.append(sent_terms)
    return final_doc

In [8]:
order_types = ['GPE', 'NORP', 'PERSON']
test = process_text_english(ar_df, order_types)
test

['number 1772 thirty-seventh year monday 28 may 1992 price 40 mm 52 e m 2 quot they except national charter engineer gamal abdel nase o bashmehndg gap us detailed drawing villa ', '3 clarity vision goals follow-up constantly avoid emotional drift sub-paths move away national struggle threaten part the three items define basic national claim also principles revolutionary philosophy revolutionary theory they define principles general strategic plan define principles detailed plan ie tactic he fall c principles extend partial details applications charter reached definition principles beyond 00 reached distant parts for example charter production management happens said quot it necessary us know written word basket among easy memorize future it also completes important link connection idea experiment those responsible national action write ideas front responsible implementation etc this principle although principle concerning details labor administration another principle organization must

['',
 '',
 'british kamal al-din hassin cairo nkrumah british british british beirut turks ataturk » cairo washington al yousif ',
 'arab american arab arab american al-yousef american arab arab america germany japan germany germany canada india arab arab arab al-yusuf al-yusuf ',
 'cairo american missouri american cairo washington arab cuba cuban american cairo british germany cairo british kuwait american cairo paris cairo cairo al-youssef ',
 '',
 'al nader • abdul-asmar al yousef ',
 '',
 'karl karl marx al-yusuf ',
 'ahmad daoud arabs mahmoud al-nallah al-hafiz falah al-atta al yousuf al yousuf mohamed ali ahmed al yousef mohamed al-yusuf ',
 'italian cairo alexandria czech cairo al-aini al-ayni ',
 'al-khayan arab ras sal ',
 'al-youssef joseph al el youssef 1771 ',
 'mohamed ali ahmed al-hai al-atta mahmud hussein » bahri al-sayed al-yusuf al-atta ',
 'albanians al-hayy al-atta al-hayy al-atta italians al yusuf ',
 '',
 'kurds ',
 'mustafa quot » st mustafa shand mustafa fatima 